In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('adel.xlsx', sheet_name=1, index_col=0)

In [3]:
df.head()

v1     v2  v3      v4     v5       v6      v7    v8  v9  v10    ...     \
ID                                                                   ...      
1    43   4205  20  210.25  97.79  3442.75   40.08  1.19  21   43    ...      
2   143   5344  26  205.54  37.37  2420.70   76.96  2.30  35   72    ...      
3    77   4760  24  198.33  61.82  2727.50   39.62  2.36  36   68    ...      
4   723  24186  49  493.59  33.45  7241.18  273.80  3.77  77  129    ...      
5   238   9549  39  244.85  40.12  3592.97   86.61  4.10  55   94    ...      

       v16    v17    v18     v19       v20   v21   v22    v23  v24       v25  
ID                                                                            
1   269.87  60.66  19.83  172.13  4.003023  1.00  98.0  100.0    8  6.722689  
2   221.73  58.26  23.83   93.10  0.651049  0.96  86.4   97.9    9  3.910000  
3   256.38  55.42  23.92  113.64  1.475844  1.17  91.7  100.0   16  6.780000  
4   349.13  87.00  45.40  147.77  0.204385  0.73  55.8   69.6   24  6.370000  
5   225.29  66.68  28.20   92.12  0.387059  0.74  78.2   94.1   18  4.390000  

[5 rows x 25 columns]

In [27]:
def get_correlation_indices(df, threshold):
    """
    Returns the indices where the Pearson correlation coefficients are
    above the given threshold in the given dataframe.
    """
    pearsonr = np.corrcoef(df, rowvar=False)
    indices = np.where(pearsonr.reshape(len(df.columns) ** 2) > threshold)[0]
    
    # The two conditions here make sure we do not include diagonal elements
    # (trivial), and that we include each pair only once.
    indices_2d = [[x // 25, x % 25] for x in indices if x // 25 != x % 25 and x // 25 < x % 25]
    
    # Display the correlations in a matrix to make it look easier
    index = 0

    for i in range(25):
        count = 0
        for j in range(25):
            if index < len(indices_2d) and i == indices_2d[index][0] \
                and j == indices_2d[index][1] and i < j:
                print('*', end=' ')
                index += 1
                count += 1
            elif i > j:
                print(' ', end=' ')
            else:
                print('-', end=' ')
        print(' ', count)
    
    return indices_2d

In [5]:
np.array(df)[indices_2d[0][0]][indices_2d[0][1]]

NameError: name 'indices_2d' is not defined

In [28]:
df.columns

Index(['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11',
       'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21',
       'v22', 'v23', 'v24', 'v25'],
      dtype='object')

In [29]:
indices = get_correlation_indices(df, 0.8)

- - - - - - * - - - - - - - - - - - - - - - - - -   1
  - - * - * - - * * - - - * - * - * * - - - - * -   9
    - - - - - - - - - - - - - - - - - - - - - - -   0
      - - * - - * * * - * * * * * - * - - - - * *   12
        - - - - - - - - - - - - * - - - - - - - -   1
          - - - * * - - * * - * * * * - - - - - *   9
            - - - - - - - - - - - - - - - - - - -   0
              - - - - - - - - - - - - - - - - - -   0
                - * * * * * * * * * - - - - - - -   9
                  - * * * * * * * * * - - - - - -   9
                    - - * * * * * * - - - - - - -   6
                      - * - - * - * - - - - - - -   3
                        - - * * * * * - - - - - -   5
                          - * * * - - - - - - * -   4
                            - * * * - - - - - - -   3
                              - * * * - - - - * *   5
                                - * * - - - - * -   3
                                  - - - - - - - -   0
                           

## Use above for OLS model

In [7]:
import statsmodels.api as sm

In [31]:
def fit_and_display(output, inputs, constant=True):
    """
    Fits an OLS model with the given output and inputs and shows the
    result summary.
    
    Args:
        output: int, the DV
        inputs: list of int, the IVs. Both args are 1-based indexed.
    """
    Y = df['v' + str(output)]
    X = df[['v' + str(x) for x in inputs]]
    
    if constant:
        X = sm.add_constant(X)
    
    model = sm.OLS(Y, X)
    results = model.fit()
    print(results.summary())

In [46]:
# Since v4 has the most correlations, may be a good candidate for a DV
fit_and_display(4, [6, 9, 11, 13, 14, 17, 19, 24, 25], False)

                            OLS Regression Results                            
Dep. Variable:                     v4   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                     2429.
Date:                Mon, 05 Nov 2018   Prob (F-statistic):               0.00
Time:                        09:51:23   Log-Likelihood:                -2143.2
No. Observations:                 618   AIC:                             4304.
Df Residuals:                     609   BIC:                             4344.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
v6            -0.0065      0.002     -2.775      0.0